# Logistische Regression auf dem Titanic Datensatz

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
%matplotlib inline
import pltdefaults as plot


## Hole Daten

Der Datensatz stammt von [Kaggle](https://www.kaggle.com/sureshbhusare/titanic-dataset-from-kaggle).

Er beinhaltet Daten zu den über 1.300 Passagieren der tragischen Jungfernfahrt. Die Daten der über 900 Mann starken Besatzung sind nicht enthalten.

Die Spalten haben folgende Bedeutung:

Variable	|Definition	|Key
:---|:---|---
Survival	|Survival - Label	|0 = No, 1 = Yes
Pclass	|Ticket class	|1 = 1st, 2 = 2nd, 3 = 3rd
Sex	|Sex	| male, female
Age	|Age in years|	
SibSp	|# of siblings / spouses aboard the Titanic	|
Parch	| # of parents / children aboard the Titanic	|
Ticket	|Ticket number	|
Fare	|Passenger fare	|
Cabin	|Cabin number	|
Embarked	|Port of Embarkation	|C = Cherbourg, Q = Queenstown, S = Southampton
Name|Name des Passagiers |


In [ ]:
train = pd.read_csv('./data/titanic/train.csv')

## Ein erster Blick auf die Daten

`shape` sagt uns die Dimensionen des Datensatzes: Wieviele Datensätze (Zeilen) und wieviele Features (Spalten) haben wir?

In [ ]:
print(train.shape)

Mit `head` oder `tail` können wir uns die ersten bzw. letzten paar Datensätze anschauen:

In [ ]:
train.head()

In [ ]:
train.tail()

`info` gibt uns ein paar Informationen zu den Datentypen der Spalten:

In [ ]:
train.info();

`describe` liefert uns einige Kennzahlen zur statistischen Verteilung der Daten:

In [ ]:
train.describe(exclude='O')

Wir konnten aus `train.info()` schon ablesen, dass zu "Age", "Embarked" und "Cabin" weniger non-null Werte als 891 angezeigt werden. Zu diesen Null-Werten müssen wir noch überlegen, wie wir damit umgehen.

Hier können wir uns ein Bild davon machen, wo die Null-Werte auftreten:

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data=train.isnull(), ax=ax);

## Exploratory Data Analysis

... wie sehen unsere Daten eigentlich aus?



Plotten wir mal eine Verteilung:

In [ ]:

f, axs = plt.subplots(2, 4, figsize=(20, 7), sharex=False)

showColumns = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

r = 0
c = 0
for col in showColumns:
    if col == 'Age' or col == 'Fare':
        sns.distplot(train[col], ax=axs[r, c])
    else:
        sns.countplot(train[col], ax=axs[r, c])
        
    if col == 'Fare':
        axs[r, c].xaxis.set_major_locator(ticker.MultipleLocator(100))
        axs[r, c].xaxis.set_minor_locator(ticker.MultipleLocator(10))
        axs[r, c].xaxis.set_major_formatter(ticker.ScalarFormatter())
    if col == 'Age':
        axs[r, c].xaxis.set_major_locator(ticker.MultipleLocator(10))
        axs[r, c].xaxis.set_minor_locator(ticker.MultipleLocator(5))
        axs[r, c].xaxis.set_major_formatter(ticker.ScalarFormatter())
    c = c + 1
    if (c > 3):
        r = 1
        c = 0


### Correlation

In der Statistik misst der Korrelationskoeffizient $\rho$ die Stärke und Richtung einer linearen Beziehung zwischen zwei Variablen.

Ab einem Absolutwert größer 0.5 sollte man die Korrelation betrachten.

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.corr(), annot=True, center=0, square=True);

Die Korrelation zwischen Fare und Pclass ist jetzt nicht überraschend. aber ggf. könnte es Sinn machen Fare zu droppen um Kollinearität zu vermeiden.

## Daten Aufbereitung


### Fehlende Werte

Die Spalte Cabin droppen wir, da uns zu viele Werte fehlen.

Die zwei fehlenden Werte in der Spalte Embarked können wir einfach mit dem häufigsten Hafen belegen: 'S'.

Die Spalte "Age" hat einige fehlende Einträge. Wir haben die Alternativen entweder die Spalte auch zu droppen, 
dann verlieren wir allerdings einige Information, oder die fehlenden Werte aufzufüllen.

Ein Ansatz dazu, ist zu schauen, wo die höchste Korrelation zu "Age" besteht und dies zum Auffüllen zu verwenden:
"Pclass" ist am höchsten korreliert mit "Age".

In [ ]:
age_group = train.groupby("Pclass")["Age"]
print(age_group.median())

### Data Pipeline

Wir packen die ganzen Umformungen in eine Funktion, da wir sie auch für den Test-Datensatz brauchen.

In [ ]:
def data_pipeline(data, scaler=None):
    X = data.copy()
    y = X['Survived']
    
    # Spalten droppen, die wir nicht weiter betrachten
    # PassengerId, Name und Ticket scheinen auf den ersten Blick wenig relevant zu sein, 
    # also droppen wir sie für unseren ersten Ansatz.
    # Cabin ist nur zu einem geringen Teil gefüllt, und eine Methode zur Befüllung der 
    # leeren Daten liegt nicht auf der Hand. Also droppen wir Cabin auch:
    X.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1, inplace=True)
     
    # Der Altersmedian unterscheided sich zwischen den Klassen signifikant, also setzen wir 
    # diesen für die fehlenden Werte ein
    X.loc[X.Age.isnull(), 'Age'] = X.groupby('Pclass').Age.transform('median')
    
    # Für die zwei fehlenden Ausgangshäfen nehmen wir einfach den häufigsten Wert an - "S"
    X.Embarked = X.Embarked.fillna('S')
    
    # Skalierung der Daten
    if not scaler:
        scaler = StandardScaler().fit(X[['Age', 'Fare']])
    X[['Age', 'Fare']] = scaler.transform(X[['Age', 'Fare']])
    
    ### One-hot Encoding "Sex" und "Embarked"
    # Pandas hat eine sehr nützliche Funktion, die kategorische Variablen One-Hot encoded und 
    # den Dataframe gleich entsprechend umwandelt - diese heißt `get_dummies`.
    # Der Name rührt daher, dass mit dem On-Hot Encoding neue Spalten (= neue Variablen) 
    # entstehen, die "Dummy-Variablen" genannt werden - da sie in gewissem Sinne ja keine 
    # "echten" Variablen sind.
    # Bei Scikit-Learn leistet dasselbe der `OneHotEncoder`.
    return scaler, pd.get_dummies(X, drop_first=True), y

### Aufbereitung der Daten über die Pipeline

In [ ]:
scaler, X, y = data_pipeline(train)
scaler.mean_

Wie sehen unsere aufbereiteten Daten jetzt aus?

In [ ]:
X.info()

In [ ]:
X.describe()

Unsere Altersverteilung sieht jetzt so aus:

In [ ]:
plt.figure(figsize = (16, 8))

rescaled = scaler.inverse_transform(X[['Age', 'Fare']])
sns.distplot(rescaled[:,0])
plt.title("Age Histogram")
plt.xlabel("Age")
plt.show()

## Training vorbereiten


Wir splitten unsere Daten noch in ein Training- und ein Test-Set im Verhältnis 80:20.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Logistische Regression

Trainiere das Modell

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
results = model.fit(X_train, y_train)
results

... und bewerte das Ergebnis ...

**Confusion Matrix:** Die Confusion-Matrix $C$ ist definiert durch: $C_{i,j}$ ist die Anzahl der Beobachtungen der wahren Gruppe $i$, die als zur Gruppe $j$ vorhergesagt werden.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print("\t'false'\t'true'")
print("false\t  {}\t  {}".format(tn, fp))
print("true\t  {}\t  {}".format(fn, tp))

## Bewertung des Modells

### Kennzahlen zur Bewertung

**Precision** ist das Verhältnis $\frac{tp}{tp + fp}$, wobei $tp$ die Anzahl der echten positiven und $fp$ die Anzahl der falschen positiven Werte ist. Intuitiv ist Precision die Fähigkeit des Klassifikators, eine negative Probe nicht als positiv zu kennzeichnen.

**Recall** ist das Verhältnis $\frac{tp}{tp + fn}$, wobei $fn$ die Anzahl der falschen Negativen ist. Intuitiv ist Recall die Fähigkeit des Klassifikators, alle positiven Proben zu finden.

Der **F-1-Score** kann als ein gewichteter harmonischer Mittelwert von Precision und Recall interpretiert werden, wobei ein F-Beta-Score seinen besten Wert bei 1 und den schlechtesten Wert bei 0 erreicht.

$f1=2\times\frac{Precision \times Recall}{Precision + Recall}$

**Support** ist die jeweilige Anzahl der Vorkommnisse der wahren Labels.

**weighted avg** gewichtet den Durchschnitt mit der Häufigkeit der Klassen.

In [ ]:
print(classification_report(y_test, predictions, target_names=['Nicht überlebt', 'Überlebt']))

### Interpretation des Modells

Schauen wir uns Intercept und Koeffizienten an

In [ ]:
print('Intercept/Bias: {}'.format(model.intercept_))

coef_dict = sorted(list(zip(X.columns.tolist(), model.coef_.ravel())), key=lambda tup: -abs(tup[1]))
for tup in coef_dict:
    print(tup)

Das Geschlecht ist mit Abstand der am stärksten eingehende Faktor, gefolgt von der Klasse.

## Übung

Versuchen Sie das Modell zu verbessern, indem Sie z.B. die Familiengröße verwenden oder aus dem Namen den Titel extrahieren:

In [ ]:
print(set([t.split(',')[1].split('.')[0] for t in train.Name]))